# Signal processing course 2018/2019-1 @ ELTE
# Assignment 1
## 09.17.2018

## Task 4
### b) Find the closest neighbour of each data point

We should define a smart distance calculation method, as this step is the most memory-intensive in the naive approach. It is also necessary to utilize the vectorization capabilities of `numpy`.

Here, I'll compare the **naive method**, where the Euclidean distance between each point-pairs are calculated, and a **smart approach**, where I'll use K-D Tree, a form of spatial indexing, used in a multitude of numerical problems.

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(rc=custom_settings)

#### Generate data points according to the uniform distribution

In [ ]:
# Number of data points
N = 1000

# Generate arrays for containing x and y coordinates
xi = np.random.random(size=N)
yi = np.random.random(size=N)

# Coordinate vectors
data = np.c_[xi, yi]

### 1. Naive approach

In [ ]:
def closest_neighbour(data):
    # `data.shape[0]` should be equal to `N`
    nbour = np.ones(data.shape[0], dtype=int)
    dists = np.ones(data.shape[0])
    # Iterate over all points (vectors) in the data set
    for i, vi in enumerate(data):
        # Subtract all vectors from this one, except itself
        # (THIS IS AN EXAMPLE OF USING THE VECTORIZATION IN NUMPY!)
        # We use `axis=0`, because we want to delete an entire vector
        data_sub = vi - np.delete(data, i, axis=0)
        # Calculate the norm of each subtracted vector
        # (THIS IS AN EXAMPLE OF USING THE VECTORIZATION IN NUMPY!)
        # We use `axis=1`, because we want to norm for each vector
        d = np.linalg.norm(data_sub, axis=1)
        # Get the index of the closest neighbour
        min_idx = np.argmin(d)
        if min_idx >= i:
            min_idx += 1  # Adjust index because of the removed element
        nbour[i] = min_idx
        # Get the distance of the closest neighbour
        dists[i] = np.min(d)
    return nbour, dists

In [ ]:
%%time
nbour, dists = closest_neighbour(data)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')

i = 100
ax.scatter(*np.delete(data, [i, nbour[i]], axis=0).T, color='0.3')
ax.scatter(*data[i].T, label='Target point', color='tab:red')
ax.scatter(*data[nbour[i]].T, label='Closest neighbour', color='tab:green')

ax.legend(fontsize=10)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(data=dists, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Distance to closest neighbour', fontsize=14)
ax.set_ylabel('Density', fontsize=14)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.histplot(data=dists, color='0.3', kde=True, ax=ax)

ax.set_xlabel('Distance to closest neighbour', fontsize=14)
ax.set_ylabel('Count', fontsize=14)

plt.show()

### 2. Smart approach with K-D Tree (WIP)

#### Implement K-D Tree

In [ ]:
class KDNode:
    def __init__(self, point, axis, left=None, right=None):
        self.point = point  # the point at the node
        self.axis = axis    # axis of split, x=0, y=1, z=2, etc.
        self.left = left    # left subtree
        self.right = right  # right subtree

def build_kdtree(points, depth=0):
    n = len(points)
    if n == 0:
        return None

    axis = depth % 2  # alternating between x and y axis
    sorted_points = sorted(points, key=lambda point: point[axis])
    median_index = n // 2

    return KDNode(
        point=sorted_points[median_index],
        axis=axis,
        left=build_kdtree(sorted_points[:median_index], depth + 1),
        right=build_kdtree(sorted_points[median_index + 1:], depth + 1)
    )